In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

# 1. Veri Hazırlığı
# PPG verisini yükle (veri dosyasının yolunu güncelleyin)
data = pd.read_csv('ppg_dataset.csv')

# Veri ön işleme
X = data.drop('label', axis=1).values
y = data['label'].values

# Veriyi normalleştir
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Veriyi yeniden şekillendir (1D veriyi 2D veriye dönüştür)
X = X.reshape(X.shape[0], X.shape[1], 1)

# Veriyi eğitim ve test setlerine ayır
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. CNN Modelini Oluşturma
model = Sequential([
    Conv1D(64, 3, activation='relu', input_shape=(X_train.shape[1], 1)),
    MaxPooling1D(2),
    Conv1D(128, 3, activation='relu'),
    MaxPooling1D(2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

# Modeli derle
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 3. Modeli Eğitme
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

# 4. Modeli Değerlendirme
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

print(classification_report(y_test, y_pred_classes))
print(confusion_matrix(y_test, y_pred_classes))

# Eğitim ve doğrulama kaybı/grafiği çizimi (isteğe bağlı)
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='training loss')
plt.plot(history.history['val_loss'], label='validation loss')
plt.legend()
plt.show()

plt.plot(history.history['accuracy'], label='training accuracy')
plt.plot(history.history['val_accuracy'], label='validation accuracy')
plt.legend()
plt.show()
